#### Exceder la capacidad de la ventana de contexto
Propósito del ejemplo  <br>
Mostrar el tipo de error que arroja Python cuando se excede la capacidad de la ventana de contexto de un LLM. 


In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("directorio/WorldBank.pdf")
pages = loader.load()
conteo = 0
for i in range(len(pages)):
    conteo = conteo + len(pages[i].page_content.split())
print("Palabras totales -\n", conteo)

In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()      # Habilitar la API
doc = []           # Crear lista vacia
for i in range(len(pages)):
    doc.append(pages[i].page_content)        # Reunir los textos de cada pagina
    
llm = ChatGroq(model="llama3-8b-8192") # Meta      # Instanciar el LLM
llm.invoke(doc)                         # Invocar el LLM con el documento completo

#### CharacterTextSplitter - Ejemplo 1
Propósito del ejemplo  <br>
Implementar el particionador CharacterTextSplitter, el más sencillo. En este primer ejemplo <br>
se usa un texto corto para apreciar mejor el funcionamiento.


In [ ]:
! pip install langchain-text-splitters

In [17]:
# Se instancia CharacterTextSplitter. Requiere un separador.
# Por defecto el separador es el Fin de linea. Acepta otros.
from langchain_text_splitters import CharacterTextSplitter

texto = 'abcdefghijklmnopqrstuvwxyz'
c_splitter = CharacterTextSplitter(
    chunk_size=14,
    chunk_overlap=0,
    separator = 'g',
)
# Se aplica CharacterTextSplitter
c_splitter.split_text(texto)

['abcdef', 'hijklmnopqrstuvwxyz']

#### CharacterTextSplitter - Ejemplo 2
Propósito del ejemplo <br>
Implementar el particionador CharacterTextSplitter con un texto largo.


In [19]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("./directorio/ConstitucionArgentina.txt",
                     encoding='utf-8')
pages = loader.load()

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=300,
    chunk_overlap=0,
)
docs = text_splitter.split_documents(pages)
print(len(docs))
docs[0].page_content

Created a chunk of size 618, which is longer than the specified 300
Created a chunk of size 301, which is longer than the specified 300


4


'La Constitución de la Nación Argentina, que rige actualmente a la República Argentina, fue aprobada por una asamblea constituyente integrada por representantes de trece provincias, hecha en la ciudad de Santa Fe en el año 1853. El propósito de la Constitución de 1853 fue poner fin al ciclo de las guerras civiles y sentar las bases de la «unión nacional» mediante un régimen republicano y federal. Antes de esta Constitución hubo dos intentos constituyentes en 1819 y 1826, que no prosperaron por carecer de consenso entre las provincias. Con posterioridad otras diez provincias y una ciudad integraron la federación.'

#### RecursiveCharacterTextSplitter - Ejemplo 1
Propósito del ejemplo <br>
Implementar el particionador RecursiveCharacterTextSplitter con un texto corto.


In [1]:
# Se instancia RecursiveCharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Se trata de particionar esta cadena de 26 caracteres
texto = 'abcdefghijklmnopqrstuvwxyz'

r_splitter = RecursiveCharacterTextSplitter(
# Se define el tamaño de los chunks y el grado de solapamiento
    chunk_size=8,
    chunk_overlap=2
)

r_splitter.split_text(texto)

['abcdefgh', 'ghijklmn', 'mnopqrst', 'stuvwxyz']

#### RecursiveCharacterTextSplitter - Ejemplo 2
Propósito del ejemplo  <br>
Implementar el particionador RecursiveCharacterTextSplitter con un texto largo.


In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

path = "./directorio/ConstitucionArgentina.txt"
with open(path,'r', encoding="UTF-8") as f:
    texto = f.read()
    
r_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", 
                 "\n", 
                 " " ], 
    chunk_size=500,
    chunk_overlap=0
)
chunks = r_splitter.create_documents([texto])
print(len(chunks))
for i in range(len(chunks)):
    print("Numero de chunk:", i+1, "\n", chunks[i].page_content)
    print()

6
Numero de chunk: 1 
 La Constitución de la Nación Argentina, que rige actualmente a la República Argentina, fue aprobada por una asamblea constituyente integrada por representantes de trece provincias, hecha en la ciudad de Santa Fe en el año 1853. El propósito de la Constitución de 1853 fue poner fin al ciclo de las guerras civiles y sentar las bases de la «unión nacional» mediante un régimen republicano y federal. Antes de esta Constitución hubo dos intentos constituyentes en 1819 y 1826, que no prosperaron por

Numero de chunk: 2 
 carecer de consenso entre las provincias. Con posterioridad otras diez provincias y una ciudad integraron la federación.

Numero de chunk: 3 
 El texto ha sido reformado varias veces, estando vigente la redacción resultante de la reforma de 1994. En varias oportunidades el orden constitucional fue dejado sin efecto por golpes de Estado, el último de los cuales impuso una dictadura que se mantuvo en el poder hasta el 10 de diciembre de 1983.

Numero de c